In [ ]:
!cp drive/MyDrive/CI-Final-Datasets/train.csv ./
!cp drive/MyDrive/CI-Final-Datasets/CI_test.csv ./

In [ ]:
pip install hazm

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from hazm import Normalizer
import nltk
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
train_data = pd.read_csv('train.csv',usecols=['comment', 'recommend'])
train_data['recommend_numeric'] = train_data['recommend'].factorize()[0]
train_data.head()

,comment,recommend,recommend_numeric
0,سلام، قبل اینکه نظرم رو بگم میخواستم به یک موض...,recommended,0
1,گیره های فلزی خیلی سخت تا میشوند و لذا حوله را...,not_recommended,1
2,سلام دوستان،،_x000D_\nمنم مثه بعضی از دوستان ق...,recommended,0
3,من چند سالی هست که این اسپیکرو خریدم و واقعا ح...,recommended,0
4,نزدیک به دو ساله که این هارد رو خریدم. استفاده...,recommended,0


In [ ]:
stopword = (open('drive/MyDrive/CI-Final-Datasets/stopwords.txt',"r").read().split('\n'))

In [ ]:
normalizer = Normalizer()
train_data['comment'] = train_data.apply(lambda row :normalizer.normalize(str(row['comment'])),axis=1)

In [ ]:
train_data ['comment'] = train_data.apply(lambda row:" ".join(w for w in nltk.wordpunct_tokenize(row['comment']) if w not in stopword),axis = 1)
train_data

,comment,recommend,recommend_numeric
0,سلام ، نظرم رو بگم میخواستم موضوع مهم اشاره کن...,recommended,0
1,گیره ‌ های فلزی سخت میشوند لذا حوله سخت می ‌ ت...,not_recommended,1
2,سلام دوستان ،، _x۰۰۰D_ منم مثه بعضی دوستان خری...,recommended,0
3,سالی اسپیکرو خریدم واقعا حرف نداره _x۰۰۰D_ برن...,recommended,0
4,نزدیک ساله هارد رو خریدم . سنگینی ازش نداشتم ی...,recommended,0
...,...,...,...
48077,کاملابی کیفیت ، قیمتی اصلا ارزش خرید نداره,not_recommended,1
48078,عالیییییییییییییییییییی,recommended,0
48079,اندازه بالش کوچک بدرد بچه ۲ ساله میخوره پولتون...,not_recommended,1
48080,تمیز زیبا … راضیم,recommended,0


In [ ]:
test_data = pd.read_csv('CI_test.csv',usecols=['id', 'comment'])
test_data.head()

,id,comment
0,1,با توجه به قدرت خنک کنندگی حداقل ۲ ساعت زمان ل...
1,2,بسیار خوش بو ولی ماندگاری در حد یک ساعت ، با ا...
2,3,اصلا تأثیری نداشت
3,4,کیفیت بدنه خیلی خوبی داره._x000D_\nوقتی در جای...
4,5,دو ساله استفاده میکنم، واقعاً هیچ کجا برام کم ...


In [ ]:
normalizer = Normalizer()
test_data['comment'] = test_data.apply(lambda row :normalizer.normalize(str(row['comment'])),axis=1)

In [ ]:
test_data ['comment'] = test_data.apply(lambda row:" ".join(w for w in nltk.wordpunct_tokenize(row['comment']) if w not in stopword),axis = 1)
test_data

,id,comment
0,1,توجه قدرت خنک کنندگی حداقل ۲ ساعت لازم داره یک...
1,2,خوش بو ماندگاری حد ساعت ، قیمت اصلا ابدا ارزش ...
2,3,اصلا تأثیری نداشت
3,4,کیفیت بدنه خوبی داره . _x۰۰۰D_ نامناسبی قرار م...
4,5,ساله میکنم ، واقعا برام کم نداشته قدرته جلوی ا...
...,...,...
4995,4996,دی جی کالا خریدم عالیه . _x۰۰۰D_ پیشنهاد میکنم...
4996,4997,خوبه . _x۰۰۰D_ اندازه هاش کاربردی عالیه
4997,4998,چرب کردن مو باعث جلوگیری سفیدی زودرس پیری مو م...
4998,4999,بعداز ماه زرد میشه میره


In [ ]:
dic={'recommended':0,'not_recommended':1}
cid = {1:'not_recommended' , 0:'recommended'}

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True,min_df=30, norm='l2', ngram_range=(1, 2))
features = vectorizer.fit_transform(train_data['comment'].values.astype('U'))
labels = train_data['recommend_numeric']
features

<48082x4465 sparse matrix of type '<class 'numpy.float64'>'
	with 801129 stored elements in Compressed Sparse Row format>

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=3, random_state=0)

In [ ]:
model.fit(features, labels)

GradientBoostingClassifier(learning_rate=1.0, random_state=0)

In [ ]:
test_features = vectorizer.transform(test_data['comment'].values.astype('U'))
test_features

<5000x4465 sparse matrix of type '<class 'numpy.float64'>'
	with 83619 stored elements in Compressed Sparse Row format>

In [ ]:
y_prediction = model.predict(test_features)

In [ ]:
prediction =[]
for recommend_numeric in y_prediction:
  prediction.append(cid[recommend_numeric])

In [ ]:
result = pd.DataFrame({
        "id": test_data["id"],
        "recommend": prediction
    })

In [ ]:
result.head(20)

,id,recommend
0,1,recommended
1,2,not_recommended
2,3,not_recommended
3,4,recommended
4,5,recommended
5,6,recommended
6,7,recommended
7,8,recommended
8,9,recommended
9,10,recommended


In [ ]:
result.to_csv('submission.csv',index=False)